# Importe de vulnerabilidades
Se importan las vulnerabilidades publicadas en la *National Vulnerability Database (NVD)* de los años 2022 y 2023

In [1]:
import requests     #Realizar peticiones
import gzip         #Compresión y descompresión
import json         #Lectura del formato json
from tqdm import tqdm   #Seguimiento de bucles

url_fmt = 'https://nvd.nist.gov/feeds/json/cve/1.1/nvdcve-1.1-{}.json.gz'   #URL a la fuente de vulnerabilidades

cve_items = list()  #Lista dónde se guardará cada vulnerabilidad

#Agregar las vulnerabilidades de 2022 y 2023 a la lista cve_items
for year in tqdm(range(2022, 2023)):
    content = requests.get(url_fmt.format(year)).content
    json_data = gzip.decompress(content)
    data = json.loads(json_data)
    cve_items += data['CVE_Items']

100%|██████████| 1/1 [00:07<00:00,  7.62s/it]


### Función para asignar el tipo de ataque a cada vulnerabilidad
Dado que la NVD no proporciona información sobre el tipo de ataque, he creado una lista que incluye algunos de los tipos de ataque más comunes. Esta lista se organiza de menor a mayor severidad según mi criterio. Como resultado, la función devuelve un elemento de la lista con una probabilidad de ocurrencia de mayor a menor.

In [2]:
#Asignación de tipos de ataques
def Set_Attack_Type():
    #Tipos de ataques comunes
    Attack_Types = [
        'Brute Force Attacks',
        'Man-in-the-Middle (MitM) Attacks',
        'Malware (Viruses, Trojans, Worms)',
        'Cross-Site Scripting (XSS)',
        'SQL Injection',
        'Distributed Denial of Service (DDoS)',
        'Advanced Persistent Threats (APTs)'
    ]
    weights = [1 / (i + 1) for i in range(len(Attack_Types))]   #Pesos de menor a mayor probabilidad
    return random.choices(Attack_Types, weights=weights)[0]     #Retorno del tipo de ataque

# Generación del conjunto de datos

Mi objetivo es generar un conjunto de datos similar al ejemplo proporcionado. Sin embargo, las vulnerabilidades importadas carecen de cierta información necesaria. Por lo tanto, he optado por completar los campos faltantes con guiones ("-"). Esta elección se basa en la observación de que el ejemplo de referencia también contiene información incompleta, la cual no es relevante para la resolución del problema en cuestión.

En cuanto a la información que considero relevante y que no se encuentra en las vulnerabilidades importadas, la completo siguiendo las pautas y posibilidades presentadas en el ejemplo de referencia. Para ello, genero información aleatoria que se ajusta a dichas pautas.

En el ejemplo, se observa que la característica 'Severity' se corresponde con el puntaje CVSS V3.1. Siguiendo las pautas para abordar el problema, donde la severidad se define con valores como 'Baja', 'Media' y 'Alta', se utiliza esta característica para asignar la severidad de la vulnerabilidad. Además, se crea una nueva columna llamada 'CVSS_Score', la cual almacena el puntaje CVSS V3.1.

In [3]:
import random   #Aleatoriedad
from datetime import datetime   #Manejo de fechas
import pandas as pd #Manejo de tablas

rows = list()   #Lista donde se concatenará las caracteristicas de cada vulnerabilidad
Exploitabilities = ['Functional', 'High', 'Unproven']   #Posibilidades de la explotabilidad

#Se recorre cada vulnerabilidad importada
for cve_item in tqdm(cve_items):
    #Comprobar si existen los siguientes elementos en las vulnerabilidades importadas
    try:
        Related_Finding = cve_item['cve']['references']['reference_data'][0]
        Severity = cve_item['impact']['baseMetricV3']['cvssV3']['baseScore']
        CWE_ids = cve_item['cve']['problemtype']['problemtype_data'][0]['description'][0]
    except:
        continue

    Related_Finding = cve_item['cve']['references']['reference_data'][0]['url'] #Asignar el correo del fabricante afectado por la vulnerabilidad
    Vulnerability_Id = cve_item['cve']['CVE_data_meta']['ID']   #Asignar las identificación de la vulnerabilidad
    Where = cve_item['cve']['CVE_data_meta']['ASSIGNER']    #Asginar el enlace a la referencia
    Stream = '-'    #Información faltante
    Specific = '-'  #Información faltante
    Description = cve_item['cve']['description']['description_data'][0]['value']    #Descripción de la vulnerabilidad
    Status = 'OPEN' #Única posibildiad observada del ejemplo
    Severity = cve_item['impact']['baseMetricV3']['cvssV3']['baseSeverity'] #Asignar severidad
    Requirements = '-'  #Información faltante
    Impact = '-'    #Información faltante
    Threat = '-'    #Información faltante
    Recommendation = '-'    #Información faltante
    External_BTS = '-'  #Información faltante
    Compromised_Attributes = '' #Información faltante
    Tags = '-'  #Inforamción faltante
    Business_Critically = '-'   #Información faltante
    Type = Set_Attack_Type()    #Asignar tipo de ataque con la función Set_Attack_Type()
    Report_Moment = cve_item['publishedDate']   #Se usa la fecha de publicación como la fecha de reporte
    Empty_Column = '-'  #Información faltante
    Age_in_days = (datetime.now().date() - datetime.strptime(Report_Moment, '%Y-%m-%dT%H:%MZ').date()).days #Días transcurridos entre el día del reporte y el día actual
    CVSSV3_string_vector = cve_item['impact']['baseMetricV3']['cvssV3']['vectorString'] #Asignar el vector de calificación
    Attack_Vector = cve_item['impact']['baseMetricV3']['cvssV3']['attackVector']    #Asignar el attackVector
    Attack_Complexity = cve_item['impact']['baseMetricV3']['cvssV3']['attackComplexity']    #Asignar el attackComplexity
    Privileges_Required = cve_item['impact']['baseMetricV3']['cvssV3']['privilegesRequired']    #Asignar el privilegesRequired
    User_Interaction = cve_item['impact']['baseMetricV3']['cvssV3']['userInteraction']  #Asignar el userInteraction
    Severity_Scope = cve_item['impact']['baseMetricV3']['cvssV3']['scope']  #Asignar el scope
    Confidentiality_Impact = cve_item['impact']['baseMetricV3']['cvssV3']['confidentialityImpact']  #Asignar el confidentialityImpact
    Integrity_Impact = cve_item['impact']['baseMetricV3']['cvssV3']['integrityImpact']  #Asignar el integrityImpact
    Availability_Impact = cve_item['impact']['baseMetricV3']['cvssV3']['availabilityImpact']    #Asignar el availabilityImpact
    Exploitability = random.choice(Exploitabilities)    #Asignar la explotabilidad con base en las posibilidades observadas en el ejemplo
    Remediation_Level = 'Official Fix' if random.random() < 0.9 else 'Unavailable'  #Asignar el nivel de remediación con base en las posibilidades observadas en el ejemplo (0% de probabilidad para 'Official Fix' y 10% de probabildiad para 'Unavailable')
    Report_Confidence = 'Confirmed' #Única posibilidad observada en el ejemplo
    CWE_ids = cve_item['cve']['problemtype']['problemtype_data'][0]['description'][0]['value']  #Asignar la identificación CWE
    Commit_Hash = '-'   #Información faltante
    Root_Nickname = '-' #Información faltante
    Group = '-' #Información faltante
    CVSSV3_score = cve_item['impact']['baseMetricV3']['cvssV3']['baseScore']    #Asignar puntaje CVSS V3.1

    #Agregar las carácteristicas a una nueva fila
    rows.append(
        [
            Related_Finding,
            Vulnerability_Id,
            Where,
            Stream,
            Specific,
            Description,
            Status,
            Severity,
            Requirements,
            Impact,
            Threat,
            Recommendation,
            External_BTS,
            Compromised_Attributes,
            Tags,
            Business_Critically,
            Type,
            Report_Moment,
            Empty_Column,
            Age_in_days,
            CVSSV3_string_vector,
            Attack_Vector,
            Attack_Complexity,
            Privileges_Required,
            User_Interaction,
            Severity_Scope,
            Confidentiality_Impact,
            Integrity_Impact,
            Availability_Impact,
            Exploitability,
            Remediation_Level,
            Report_Confidence,
            CWE_ids,
            Commit_Hash,
            Root_Nickname,
            Group,
            CVSSV3_score
        ]
    )

#Construir el conjunto de datos a partir de las filas y asignando el nombre a las respectivas columnas
df = pd.DataFrame(
    rows,
    columns=[
        'Related Finding',
        'Vulnerability Id',
        'Where',
        'Stream',
        'Specific',
        'Description',
        'Status',
        'Severity',
        'Requirements',
        'Impact',
        'Threat',
        'Recommendation',
        'External BTS',
        'Compromised Attributes',
        'Tags',
        'Business Critically',
        'Type',
        'Report Moment',
        'Empty Column',
        'Age in days',
        'CVSSV3 string vector',
        'Attack Vector',
        'Attack Complexity',
        'Privileges Required',
        'User Interaction',
        'Severity Scope',
        'Confidentiality Impact',
        'Integrity Impact',
        'Availability Impact',
        'Exploitability',
        'Remediation Level',
        'Report Confidence',
        'CWE ids',
        'Commit Hash',
        'Root Nickname',
        'Group',
        'CVSS_Score'
    ]
)

df.to_csv('dataset.csv')

100%|██████████| 24256/24256 [00:00<00:00, 43537.82it/s]
